In [ ]:
# https://stackoverflow.com/questions/23887881/how-to-repeat-pandas-data-frame

In [ ]:
# !pip install -r requirements.txt

In [1]:
import os 
from fnmatch import fnmatch
import pandas as pd
import altair as alt
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from vega_datasets import data

alt.renderers.enable('default')

RendererRegistry.enable('default')

In [2]:
##
# 
##
def find_data(pattern= "*.csv", root = os.getcwd()):
    filenames = list()

    for path, subdirs, files in os.walk(root):
        for name in files:
            if fnmatch(name, pattern):
                filenames.append(os.path.join(path,name))
    
    if not filenames:
        print("No files found")
        return 
    else:
        return filenames

In [3]:
data_list = find_data()
data_list[3]

'c:\\Users\\Michael\\Desktop\\CMU\\CMU Classwork\\Spring 2021\\839\\Final Project\\Final_Project_code\\fp--05839-pozioma-ahamdala-akintayj-mfinch\\scs\\python\\data\\mrc_table3.csv'

In [4]:
df1 = pd.read_csv(data_list[0])
df2 = pd.read_csv(data_list[2])
df3 = pd.read_csv(data_list[3])

In [5]:
def Mag_dis(MR):
    p_p = np.sum(MR,axis = 0)
    p_k = np.sum(MR, axis = 1)
    return p_p, p_k

def Correlation(MR):
    s = MR.shape
    p = np.array(list(range(s[1]))) + 1   
    k = np.array(list(range(s[0]))) + 1 
    p_p, p_k = Mag_dis(MR)

    up = p@p_p
    up2 = (p*p)@p_p
    uk = k@p_k
    uk2 = (k*k)@p_k
    upk = ((k[:,None]@(p[:,None]).T)*MR).sum()

    sk2 = uk2 - uk*uk
    sp2 = up2 - up*up
    skp = upk - up*uk
    rkp = skp/np.sqrt(sk2*sp2)
   
    return np.array([[sk2, skp],[skp, sp2]]), np.array([[1, rkp],[rkp, 1]])

# def Mutual_info(MR):
#     p_p, p_k = Mag_dis(MR)
    
#     I = MR*(np.log2(MR/(p_p*p_k)))

#     return I.sum()




In [ ]:
# https://stackoverflow.com/questions/17071871/how-to-select-rows-from-a-dataframe-based-on-column-values

In [6]:
def university_df(df, univ):
    return df.loc[df['name'] == univ]

def MR_df(dfu):
    
    Mobility_rate = np.zeros([5,5])

    for p in range(0,5):
        for k in range(0,5):
            cond_str = "kq{}_cond_parq{}".format(k+1,p+1)
            par_str = "par_q{}".format(p+1)
            MR_temp = (dfu[par_str]*dfu[cond_str])
            Mobility_rate[k,p] = MR_temp
            
    return Mobility_rate




In [7]:
k, p = np.meshgrid(range(1,6), range(1,6)) 
cmu =  "Carnegie Mellon University"
fcc = "Frederick Community College"
fit = "Florida Institute Of Technology"
mit = "Massachusetts Institute Of Technology"
UMD = "University Of Maryland System (Except University College) And Baltimore City Community College"
HU = "Howard University"
df_cmu = university_df(df3, cmu)
df_cmu = df_cmu.reset_index() 


In [8]:
df_cmu["MR"] = df_cmu.apply(MR_df, axis=1)
df_cmu["corr"] = df_cmu.apply(lambda row: Correlation(row["MR"]), axis=1)
df_cmu["Mag"] = df_cmu.apply(lambda row: Mag_dis(row["MR"]), axis=1)
df_cmu1 = df_cmu[["cohort","MR"]]
df_cmu2 = df_cmu[["cohort","Mag"]]

In [ ]:
# https://medium.com/swlh/a-deep-conceptual-guide-to-mutual-information-a5021031fad0

In [9]:
def Norm_Mutual_Info(MR):
    
    p_p, p_k = Mag_dis(MR)

    Hp = -1*p_p*np.log2(p_p) 
    Hk = -1*p_k*np.log2(p_k)
    Hpk = -1*MR*np.log2(MR)
    I = Hk.sum() + Hp.sum() - Hpk.sum() 
    return (2*I)/(Hp.sum()+Hk.sum()) 

def Jaccard_dis(MR):
    I = Mutual_info(MR)
    Hpk = -1*MR*np.log2(MR)
    
    return  1 - I/Hpk.sum()
    
def K_L_div(MR):
    p_p, p_k = Mag_dis(MR)

    Dkl = p_p*np.log2(p_p/p_k)

    return Dkl.sum()

def skewness(MR):
    p_p, p_k = Mag_dis(MR)
    T1 = 0.5*(p_p[-1]+p_p[0]) 
    T2 = (p_p[-1]-p_p[0])
    T3 = 0.5*(p_p[-2]+p_p[1]) 
    T4 = (p_p[-2]-p_p[1])
    
    return (1/3)*(list(range(1,6))*p_p).sum()-1


In [10]:
# print(df_cmu1.head(5))

# (1/3)*( np.array([0.29421377, 0.20919478,  0.22377882, 0.16188636,  0.11092634])*list(range(1,6)) ).sum() - 1

print(df_cmu1["MR"].apply(skewness))




0     0.407745
1     0.413146
2     0.433093
3     0.428281
4     0.413356
5     0.420920
6     0.423880
7     0.422973
8     0.414635
9     0.459321
10    0.452633
11    0.447534
Name: MR, dtype: float64


In [11]:
# def data_plot(r1):
#     rtemp = r1["MR"].ravel()
#     k, p = np.meshgrid(range(1,6), range(1,6))  
#     plot_df = pd.DataFrame({"k":k.ravel(), "p":p.ravel(), "MR":rtemp, "cohort":r1["cohort"]})
    
#     chart = alt.Chart(plot_df).mark_rect().encode(
#         column = "cohort:O",
#         x='p:O',
#         y=alt.Y('k:O',
#         sort=alt.EncodingSortField('k', order='descending')),
#         # color = "MR:Q"
#     )

#     return chart

def gen_plot_df(r1):

    rtemp = r1["MR"].ravel()
    k, p = np.meshgrid(range(1,6), range(1,6))  
    plot_df = pd.DataFrame({"k":k.ravel(), "p":p.ravel(), "MR":rtemp, "cohort":r1["cohort"]})
    return plot_df

def gen_plot_mag(r1):

    mag_p = r1['Mag'][0].ravel()
    mag_k = r1['Mag'][1].ravel()
    k, p = list(range(1,6)), list(range(1,6)) 
    plot_df = pd.DataFrame({"k":k,'Mag_k':mag_k,"p":p,'Mag_p':mag_p, "cohort":r1["cohort"]})
        
    return plot_df


In [12]:
year = 2
source = gen_plot_df(df_cmu1.iloc[year])
source1 = gen_plot_mag(df_cmu2.iloc[year])



In [158]:
s1 = source[source["k"]>source["p"]]
s1 = s1.groupby(['p']).sum()
s1['cohort'] = source['cohort']
s1['k'] = source['k']

s2 = source[source["k"]==source["p"]]
s2 = s2.groupby(['p']).sum()
s2['cohort'] = source['cohort']
s2['k'] = source['k']

s3 = source[source["k"]<source["p"]]
s3 = -1*s3.groupby(['p']).sum()
s3['cohort'] = source['cohort']
s3['k'] = source['k']

s1["type"] = "pos"
s2["type"] = "same"
s3["type"] = "neg"

s = pd.concat([s1,s2,s3])
s = s.reset_index()
s['MR_abs']= round(abs(s['MR'])*100,2)
s


,p,k,MR,cohort,type,MR_abs
0,1,2,0.080510,1982,pos,8.05
1,2,3,0.063311,1982,pos,6.33
2,3,4,0.070316,1982,pos,7.03
3,4,5,0.080881,1982,pos,8.09
4,1,2,0.004342,1982,same,0.43
5,2,3,0.006517,1982,same,0.65
6,3,4,0.006236,1982,same,0.62
7,4,5,0.019283,1982,same,1.93
8,5,1,0.420648,1982,same,42.06
9,2,3,-0.003479,1982,neg,0.35


In [159]:


color_scale = alt.Scale(
    domain=["pos","same","neg"],
    range=["green","orange","red"]
)

color_scale1 = alt.Scale(
    range=["greens","oranges","reds"]
)

bar_chart = alt.Chart(s).mark_bar().encode(
    column = "cohort",
    x = "sum(MR):Q",
    color = alt.Color('type:N',scale=color_scale),
    tooltip=[alt.Tooltip("sum(MR_abs)"),alt.Tooltip('p')]
)



bar_chart

alt.Chart(...)

In [102]:
d_plus = alt.Chart(source[source["k"]>source["p"]]).mark_bar().encode(
    column = "cohort",
    x = alt.X('sum(MR):Q', scale=alt.Scale(domain=(0,1))),
    color = alt.Color('p:O', 
        scale=alt.Scale(
            scheme="greens",domain=(list(range(1,6)))
            ))
    )

d_same = alt.Chart(source[source["k"]==source["p"]]).mark_bar().encode(
    column = "cohort",
    x = alt.X('sum(MR):Q', scale=alt.Scale(domain=(0,1))),
    color = alt.Color('p:O', 
        scale=alt.Scale(
            scheme="oranges",domain=(list(range(1,6)))
            ))
    )



In [56]:

d_plus = alt.Chart(source[source["k"]>source["p"]]).mark_bar().encode(
    column = "cohort",
    x = alt.X("k:O", scale=alt.Scale(domain=(list(range(1,6)))) ),
    y = alt.Y('sum(MR):Q', scale=alt.Scale(domain=(0,1))),
    color = alt.Color('p:O', 
        scale=alt.Scale(
            scheme="greens",domain=(list(range(1,6)))
            ))
    )

d_same = alt.Chart(source[source["k"]==source["p"]]).mark_bar().encode(
    column = "cohort",
    x = alt.X("k:O", scale=alt.Scale(domain=(list(range(1,6)))) ),
    y = alt.Y('sum(MR):Q', scale=alt.Scale(domain=(0,1))),
    color = alt.Color('p:O', 
        scale=alt.Scale(
            scheme="oranges",domain=(list(range(1,6)))
            ))
    )

d_min = alt.Chart(source[source["k"] < source["p"]]).mark_bar().encode(
    column = "cohort",
    x = alt.X("k:O", scale=alt.Scale(domain=(list(range(1,6)))) ),
    y = alt.Y('sum(MR):Q', scale=alt.Scale(domain=(0,1))),
    color = alt.Color('p:O', 
        scale=alt.Scale(
            scheme="reds",domain=(list(range(1,6)))
            ))
    )

d_plus

alt.Chart(...)

In [13]:
k_mag_1 = alt.Chart(source).mark_bar().encode(
    column = 'cohort',
    x = alt.X("k:O"),
    y = alt.Y('sum(MR):Q', scale=alt.Scale(domain=(0,1))),
    color = alt.Color('p:O', scale=alt.Scale(scheme="tableau10"))
    )

k_mag_1 

alt.Chart(...)

In [14]:

plot_scale = alt.Scale(type="pow", exponent=0.5, scheme = "greens", nice = True)
color = alt.Color('MR:Q', scale = plot_scale)

chart = alt.Chart(source).mark_rect().encode(
        column = "cohort:O",
        x='p:O',
        y=alt.Y('k:O',
        sort=alt.EncodingSortField('k', order='descending')),
        color=color
    )
    
k_mag = alt.Chart(source1).mark_bar().encode(
    column = "cohort:O",
    x = alt.X("k:O"),
    y = alt.Y(
        "Mag_k:Q", scale=alt.Scale(domain=(0,1))
        )
    )

p_mag = alt.Chart(source1).mark_bar().encode(
    column = "cohort:O",
    x = "p:O",
    y = alt.Y(
        "Mag_p:Q", scale=alt.Scale(domain=(0,1))
        )
    )


chart

alt.Chart(...)

In [15]:
p_mag

alt.Chart(...)

In [16]:
k_mag

alt.Chart(...)

In [ ]:
# chart_list = list()
# chart_list.append(df_cmu1.apply(data_plot, axis = 1))

# chart_list = np.array(chart_list).flatten()

# # alt.renderers.enable('notebook')



In [ ]:
# rtemp = r1["MR"].ravel()

# # col = pd.MultiIndex.from_product(["k","p","MR"])
# col = ["k","p","MR", "cohort"]
# plot_df = pd.DataFrame({"k":k.ravel(), "p":p.ravel(), "MR":rtemp, "cohort":r1["cohort"]})
# plot_df


In [ ]:
# alt.Chart(plot_df).mark_rect().encode(
#     column = "cohort:O",
#     x='p:O',
#     y=alt.Y('k:O',
#       sort=alt.EncodingSortField('k', order='descending')),  
#     color="MR:Q"

# )




In [ ]:


# MR.index = date.values

# df_plot1 = pd.DataFrame(MR)

# k_p = pd.DataFrame({'k':k.ravel(),'p':p.ravel()})

# df_MR = pd.DataFrame(MR)
# # df_MR = df_MR.explode('MR')
# # df_MR.index.name = "cohort"
# # df_MR.melt(var_name="cohort")
# pd.DataFrame(data=MR.values, columns=MR.index)


In [ ]:
# # Mobility_rate = (Mobility_rate[1:2,:5])


# s = (Mobility_rate).shape

# k, p = np.meshgrid(range(1,s[0]+1), range(1,s[1]+1))        
# print(Mobility_rate)

# source = pd.DataFrame({'k': k.ravel(),'p':p.ravel(),'MR':Mobility_rate.ravel()})

# alt.Chart(source).mark_rect().encode(
#     x='p:O',
#     y=alt.Y('k:O',
#       sort=alt.EncodingSortField('k', order='descending')),  
#     color ='MR:Q'
# )



In [ ]:
# https://stackoverflow.com/questions/11409690/plotting-a-2d-array-with-mplot3d

In [ ]:
# # setup the figure and axes
# fig = plt.figure(figsize=(8, 3))
# ax1 = fig.add_subplot(121, projection='3d')


# # fake data

# x, y = p.ravel(), k.ravel()
# z = np.zeros(25)
# dx = z+1
# dy = z+1
# dz = Mobility_rate.ravel()


# ax1.bar3d(x, y, z, dx, dy, dz, shade=True)
# ax1.set_title('')


# plt.show()

In [ ]:
# c1 = Correlation(Mobility_rate)
# c2 = Correlation(Mobility_rate[:1,:5])

# print(c1[0])
# print()
# print(c1[1])
# print()
# print(c2[0])
# print()
# print(c2[1])
# print(Mobility_rate)
# print()
# print(Mag_dis(Mobility_rate))

In [ ]:
# # k is row, p is col
# df_cmu = df2.loc[df2['name'] == "Carnegie Mellon University"]

# Mobility_rate = np.zeros([5,5])

# for p in range(0,5):
#     for k in range(0,5):
#         cond_str = "kq{}_cond_parq{}".format(k+1,p+1)
#         par_str = "par_q{}".format(p+1)
#         MR_temp = (df_cmu[par_str]*df_cmu[cond_str]).to_numpy()
#         print(par_str)
#         Mobility_rate[k,p] = MR_temp
#         print(Mobility_rate)
    



In [ ]:
# df_cmu = university_df(df3, "Carnegie Mellon University")

# (df_cmu.iloc[0])["par_q1"]
